<h1 style="text-align:center;">🦜 Natural Language Processing with Transformers 🤗</h1><br><br>

<h2 style="text-align:center;">Chapter 2. Text Classification</h2><br><br>

<h4 style="text-align:center;"><b>Christopher Akiki</b></h4>

<h1 style="text-align:center;">Table of Contents</h1>
<br><br><br>
<ul>
    <li><h3>The Dataset</h3></li>
    <br>
    <li><h3>From Text to Tokens</h3></li>
    <br>
    <li><h3>Training a Text Classifier</h3></li>
</ul>

<center><img src="images/chapter02_hf-libraries.png" width=1800></center>

<h1 style="text-align:center;">🤗 Datasets</h1>

# Apache Arrow backend ➡️ Low RAM use through memmaping

In [4]:
import os
import psutil
from datasets import load_dataset


mem_before = psutil.Process(os.getpid()).memory_info().rss / (1024**2)

stack_smol = load_dataset("bigcode/the-stack-smol", split="train")

mem_after = psutil.Process(os.getpid()).memory_info().rss / (1024**2)

print(f"RAM usage when loading a {stack_smol.dataset_size / (1024**3):.3f}GB dataset: {(mem_after - mem_before)} MB")

Using custom data configuration bigcode--the-stack-smol-2e98eace392455c7
Found cached dataset json (/mnt/1da05489-3812-4f15-a6e5-c8d3c57df39e/cache/huggingface/bigcode___json/bigcode--the-stack-smol-2e98eace392455c7/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


RAM usage when loading a 2.547GB dataset: 17.34375 MB


In [6]:
stack_smol[10000]

{'content': 'nuget pack SadConsole-debug.nuspec -Symbols',
 'avg_line_length': 43.0,
 'max_line_length': 43,
 'alphanum_fraction': 0.8604651163,
 'licenses': ['MIT'],
 'repository_name': '256colour/SadConsole',
 'path': 'nuget/make-debug.bat',
 'size': 43,
 'lang': 'Batchfile'}


# Apache Arrow Backend ➡️ Fast Iteration
<br>

In [8]:
%%time
batch_size = 1000
for i in range(0, len(stack_smol), batch_size):
    batch = stack_smol[i:i + batch_size]

CPU times: user 2.84 s, sys: 0 ns, total: 2.84 s
Wall time: 2.84 s


In [9]:
stack_smol

Dataset({
    features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'],
    num_rows: 300000
})

In [10]:
%%time
batch_size = 1000
for i in range(0, len(stack_smol), batch_size):
    batch = stack_smol[i:i + batch_size]["size"]

CPU times: user 2.75 s, sys: 0 ns, total: 2.75 s
Wall time: 2.74 s


In [11]:
%%time
batch_size = 1000
for i in range(0, len(stack_smol), batch_size):
    batch = stack_smol.select(range(i,i + batch_size))["size"]

CPU times: user 468 ms, sys: 0 ns, total: 468 ms
Wall time: 466 ms


<h2 style="text-align:center;">Loading your own files</h2>
<br><br>

<table><thead><tr><th align="center">Data format</th> <th align="center">Loading script</th> <th align="center">Example</th></tr></thead> <tbody><tr><td align="center">CSV &amp; TSV</td> <td align="center"><code>csv</code></td> <td align="center"><code>load_dataset("csv", data_files="my_file.csv")</code></td></tr> <tr><td align="center">Text files</td> <td align="center"><code>text</code></td> <td align="center"><code>load_dataset("text", data_files="my_file.txt")</code></td></tr> <tr><td align="center">JSON &amp; JSON Lines</td> <td align="center"><code>json</code></td> <td align="center"><code>load_dataset("json", data_files="my_file.jsonl")</code></td></tr> <tr><td align="center">Pickled DataFrames</td> <td align="center"><code>pandas</code></td> <td align="center"><code>load_dataset("pandas", data_files="my_dataframe.pkl")</code></td></tr></tbody></table>

<h1 style="text-align:center;">🤗 Tokenizers</h1>

<center><img src="images/tokenization_pipeline.svg" width=1200></center>

In [12]:
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

sentence = "Hello how are U tday?"

In [15]:
sentence = bert_tokenizer.backend_tokenizer.normalizer.normalize_str(sentence)

In [16]:
bert_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(sentence)

[('hello', (0, 5)),
 ('how', (6, 9)),
 ('are', (10, 13)),
 ('u', (14, 15)),
 ('tday', (16, 20)),
 ('?', (20, 21))]

In [21]:
bert_tokenizer(sentence)[0].ids

[101, 7592, 2129, 2024, 1057, 14595, 4710, 1029, 102]

In [23]:
bert_tokenizer("Helllo")[0].tokens

['[CLS]', 'hell', '##lo', '[SEP]']

In [27]:
bert_tokenizer("10001")[0].tokens

['[CLS]', '1000', '##1', '[SEP]']

In [28]:
bert_tokenizer.vocab

{'bingo': 27137,
 'ideological': 17859,
 'corp': 13058,
 'vehicles': 4683,
 'cave': 5430,
 'ao': 20118,
 'instructor': 9450,
 'loss': 3279,
 'messengers': 28938,
 '##tti': 6916,
 '##cine': 16567,
 '##ang': 5654,
 'bone': 5923,
 '##oot': 17206,
 'homeless': 11573,
 'joanna': 15730,
 'proof': 6947,
 'persistence': 28297,
 'erratic': 24122,
 'horses': 5194,
 '(': 1006,
 'western': 2530,
 '1858': 8517,
 '##ʎ': 29701,
 '##ʿ': 29714,
 'elliott': 9899,
 'powell': 8997,
 'fast': 3435,
 'cbs': 6568,
 'objectives': 11100,
 'hundreds': 5606,
 'gradient': 17978,
 '##cos': 13186,
 'crusade': 16282,
 'glossy': 19504,
 'clover': 25133,
 '##ntino': 25318,
 'nervously': 12531,
 '##ps': 4523,
 'flowed': 13230,
 'international': 2248,
 'concepts': 8474,
 'episodes': 4178,
 '##行': 30471,
 '##ended': 21945,
 'analyzed': 16578,
 'classics': 10002,
 'jersey': 3933,
 'compassion': 15398,
 'subscribers': 17073,
 'combine': 11506,
 'habsburg': 17486,
 'chat': 11834,
 '##kari': 20224,
 'ho': 7570,
 'everett': 15

<h1 style="text-align:center;">Classifiying Text</h1>

<ul>
    <li><h3>Transformers as Feature Extractors</h3></li>
    <br>
    <li><h3>Fine-tuning Transformers</h3></li>
</ul>

<h1 style="text-align:center;">More Ways of Classifiying Text</h1>

<ul>
    <li><h3>Sentence Transformers as Feature Extractors ➡️ <a href="https://www.sbert.net/">(sbert.net)</a></h3></li>
    <br><br>
    <li><h3>Sentence Transformer Fine-tuning ➡️ <a href="https://hf.co/blog/setfit">(hf.co/blog/setfit)</a></h3></li>
    <br><br>
    <li><h3>Pattern-Exploiting Training ➡️ <a href="https://github.com/timoschick/pet">(github.com/timoschick/pet)</a></h3></li>
</ul>

<h1 style="text-align:center;">(Re)sources</h1>

- https://hf.co/tasks

- https://github.com/nlp-with-transformers/notebooks

- https://hf.co/docs

- https://hf.co/course

<center><a href="https://www.oreilly.com/library/view/natural-language-processing/9781098103231/"><img src="images/book_cover.png" width=400></a></center>